## Launch an Inference Server (vLLM) for the Compressed Model

This step deploys the compressed model using a vLLM inference server and exposes an OpenAI-compatible API endpoint. The server enables system-level performance evaluation using GuideLLM, allowing measurements such as latency, throughput, and time-to-first-token under realistic load conditions. These results will later be compared against the baseline established by the base model.

**Goal**: Serve the compressed model through an API to evaluate the performance impact of model compression.

**Output**: vLLM server running with the compressed model, ready to handle inference requests.

**Resources used**: 46GB L40S GPU × 1

More details on vLLM are provided in [Model_Serving_vLLM.md](Vllm_Server_README.md)

In [ ]:
import os

from utils import generate, stream

In [ ]:
# set the logging level for vLLM inference
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"

**Before starting this notebook, use `kill -9 <pid>` to kill any running processes that might consuming GPU memory.**

### vLLM config for single node

We will be using the configuration for a single-node, signle-GPU set up to launch a vLLM server for the compressed model. 

Run the following command in terminal to serve the compressed model using vLLM.

**NOTE**: 
- If your resources cannot serve the two (compressed and base) models together, make sure to stop the vLLM server for the base model(if running) before starting the compresed model server or you will get an Out Of Memory(OOM) error.

- If your system can host the two models simutalnously, keep the ``--port`` parameter different for the base and compressed models.

- The configuration used to serve the base model (in the [Base.ipynb](Base.ipynb) and the compressed model notebook) is the same other than the model name.

- Make sure you are in the parent directory of compressed_model.

```
vllm serve \
  "../Llama_3.1_8B_Instruct_int8_dynamic" \
  --host 127.0.0.1 \
  --port 8000 \
  --gpu-memory-utilization 0.6 \
  --tensor-parallel-size 1 \
  --pipeline-parallel-size 1 \
  --max-model-len 2048
```

Once the server starts, you will see something like this:

```INFO:     Started server process [37883]```\
```INFO:     Waiting for application startup.```\
```INFO:     Application startup complete.```


## A test run to see if the vLLM server is accessible
We use a helper function **generate** (defined in [utils.py](./utils.py)) to simplify sending requests to our locally-served VLLM model.

This function wraps the OpenAI-compatible Chat Completions API exposed by VLLM.

In [ ]:
# For non streaming results
response = generate(
    model="../Llama_3.1_8B_Instruct_int8_dynamic",
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8000,
    api_key="empty",
    max_tokens=512,
)
print(response)

Photosynthesis is a vital process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of glucose. This process is essential for life on Earth, as it provides the primary source of energy and organic compounds for nearly all living organisms.

The word "photosynthesis" comes from the Greek words "photo" meaning light and "synthesis" meaning putting together. During photosynthesis, plants use energy from sunlight to convert carbon dioxide and water into glucose and oxygen. This process involves several stages:

1. **Light absorption**: Light is absorbed by pigments such as chlorophyll, which is present in the chloroplasts of plant cells.
2. **Water absorption**: Water is absorbed by the roots and transported to the chloroplasts.
3. **Carbon dioxide absorption**: Carbon dioxide is absorbed from the air through small openings called stomata.
4. **Conversion of light energy**: The light energy is converted into chemical energy through

In [ ]:
# For streaming results
res = ""
for chunk in stream(
    model="../Llama_3.1_8B_Instruct_int8_dynamic",
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8000,
    api_key="empty",
    max_tokens=512,
):
    res += chunk
    print(chunk, end="", flush=True)

Photosynthesis is a vital process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of glucose. This process is essential for life on Earth, as it provides the primary source of energy and organic compounds for nearly all living organisms.

The word "photosynthesis" comes from the Greek words "photo" (light) and "synthesis" (putting together). During photosynthesis, plants use energy from sunlight, water, and carbon dioxide to produce glucose and oxygen. The overall equation for photosynthesis is:

6 CO2 (carbon dioxide) + 6 H2O (water) + light energy → C6H12O6 (glucose) + 6 O2 (oxygen)

Here's a simplified overview of the process:

1. **Light absorption**: Plants absorb light energy from the sun through specialized pigments such as chlorophyll.
2. **Water absorption**: Plants absorb water from the soil through their roots.
3. **Carbon dioxide absorption**: Plants absorb carbon dioxide from the air through small openings on the

In [ ]:
print(res)

Photosynthesis is a vital process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of glucose. This process is essential for life on Earth, as it provides the primary source of energy and organic compounds for nearly all living organisms.

The word "photosynthesis" comes from the Greek words "photo" (light) and "synthesis" (putting together). During photosynthesis, plants use energy from sunlight, water, and carbon dioxide to produce glucose and oxygen. The overall equation for photosynthesis is:

6 CO2 (carbon dioxide) + 6 H2O (water) + light energy → C6H12O6 (glucose) + 6 O2 (oxygen)

Here's a simplified overview of the process:

1. **Light absorption**: Plants absorb light energy from the sun through specialized pigments such as chlorophyll.
2. **Water absorption**: Plants absorb water from the soil through their roots.
3. **Carbon dioxide absorption**: Plants absorb carbon dioxide from the air through small openings on the

### Checking GPU vRAM
Loading the compressed model with the configuration defined in the above command will take approximately 28.5GB. It may seem surprising that a compressed 8.5 GB model consumes ~28.5 GB GPU memory. This is expected behavior in vLLM, due to how memory is allocated during inference. The main contributors are:

1. **Model Weights (~8.5 GB)**

    The size of your compressed model stored on disk (INT8, FP16, etc.). 
    Loaded once into GPU memory.
   
2. **Runtime GPU Memory (~6 GB)**

- vLLM reserves extra memory for:

- Parameter sharding

- CUDA kernels

- Attention buffers and temporary tensors

- Weight adapters and padded tensors

This adds ~4–8 GB depending on the model.

3. **KV Cache (~14 GB)**

 - Stores key/value tensors for each generated token to avoid recomputation.

- Memory grows with sequence length, model hidden size, and concurrency.

- vLLM presets a large KV cache to support batching efficiently.


4. **GPU Memory Utilization Flag (--gpu-memory-utilization)**

``--gpu-memory-utilization`` is set to 0.6, meaning vLLM can utilize 60% of the total GPU memory. In this case, we have used one 46GB LS40 GPU, 60% of 46 is approx 28.